# 画像認識：男女の判別

男女の画像から性別を判別するモデルを生成する。
- 転移学習：VGG16モデルを使用し、最後尾の畳み込み３層の重みを更新
- 生成したモデルをファイルに保存し、別途作成した顔検出プログラムにおいて性別判定に用いる

In [12]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras import optimizers

# ----- データ準備 -----

# 画像ファイル名の一覧を取得
files_male = os.listdir('./gender_data/male/')
files_female = os.listdir('./gender_data/female/')

imgs_male = []  # 男性の顔画像リスト
imgs_female = []  # 女性の顔画像リスト

# 画像を読み込み、学習モデルの入力サイズに変更して、リストに追加
for i in range(len(files_male)):
    img = cv2.imread('./gender_data/male/' + files_male[i])
    img = cv2.resize(img, (50,50))
    imgs_male.append(img)

for i in range(len(files_female)):
    img = cv2.imread('./gender_data/female/' + files_female[i])
    img = cv2.resize(img, (50,50))
    imgs_female.append(img)

# 男女の画像リストを結合し、正解ラベル（0(male), 1(female)）を作成
X = np.array(imgs_male + imgs_female)
y =  np.array([0] * len(imgs_male) + [1] * len(imgs_female))

# 男女それぞれに偏ったindexをランダムに混ぜる
random_index = np.random.permutation(np.arange(len(X)))
X = X[random_index]
y = y[random_index]

# データを学習用とテスト用に分割（8対2）
X_train = X[:int(len(X)*0.8)]
y_train = y[:int(len(y)*0.8)]
X_test = X[int(len(X)*0.8):]
y_test = y[int(len(y)*0.8):]

# 正解ラベルをone-hotベクトル（0(male), 1(female)の2クラス）に変換
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# ----- 予測モデル構築 -----

# vgg16モデルのインスタンスを生成
input_tensor = Input(shape=(50, 50, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# vgg16と連結するモデルを生成
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(BatchNormalization())  # ReLU活性化関数の出力は範囲が限定されないので正規化
top_model.add(Dropout(rate=0.5))
top_model.add(Dense(2, activation='softmax'))

# モデルの連結
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# vgg16の16層目までの重みを固定（最後尾の畳み込み３層の重みは更新）
for layer in model.layers[:15]:
    layer.trainable = False
    
# モデルをコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              #optimizer='sgd',
              metrics=['accuracy'])

# モデル構成確認
model.summary()

# ----- モデルの学習と予測評価 -----

# 学習
model.fit(X_train, y_train, batch_size=32, epochs=20)

# モデルをファイルに保存（学習済みモデルとして活用）
model.save('model_vgg16.h5', include_optimizer=True)
#model.save('model_vgg16_fls.h5', include_optimizer=False)

# テストデータによるモデルの精度評価
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# ----- 検証用 -----

# # 男女画像から性別を予測して表示
# gender = None
# for i in range(5):
#     img = cv2.imread('./gender_data/male/' + files_male[i])
#     #img = cv2.imread('./gender_data/female/' + files_female[i])  # 女性でテスト

#     # cv2のimread()で読み込んだデータは色指定順がBGRとなっているのでRGBに変更
#     b, g, r = cv2.split(img) 
#     img = cv2.merge((r, g, b))

#     # 画像を表示
#     plt.imshow(img)
#     plt.show()
    
#     # 画像サイズをモデルの入力サイズに変更する
#     img = cv2.resize(img, (50, 50))

#     # 画像をモデルの入力Shapeに合わるため、４次元ndarrayに変換：（行番, width, hight, rgb)
#     img_reshape = img.reshape((-1, 50, 50, 3))

#     # 性別を予測：0(male), 1(female)
#     pred = np.argmax(model.predict(img_reshape))
#     #pred = np.argmax(model.predict(np.array([face_img])), axis=1)
#     if pred == 0:
#         gender =  'male'
#     else:
#         gender = 'female'

#     print(gender)

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)     